In [1]:
from utils import *

In [2]:
from speedup import *

In [ ]:
_="""def get_raw_signal_with_bases(sam_data, fast5_path, reference_sequence, verbose=False):
    import warnings
    result = []

    for entry in tqdm(sam_data, total=len(sam_data), desc='Processing data'):
        fast5_signals, read_ids = read_fast5(fast5_path + entry['fast5_file_name'], 'm', get_read_ids=True)

        # Get split raw signal
        _, signals, _ = split_raw_signal(fast5_signals[find_read_id_index(read_ids, entry['read_id'])][entry['trim_offset']:], 
                                         entry['move_table'], entry['stride'])

        counter = 0
        ref_counter = entry['mapped_position_index']

        sequence = []
        ref_seq_with_deletions_insertions = []

        signal = []

        for i, (op, length) in enumerate(zip(entry['cigar_str'][1], entry['cigar_str'][0])): 
            if op == "S":  
                counter += length  

            elif op == "M":  
                sequence.extend(entry['sequence'][counter:counter + length])  
                signal.append(signals[counter])
                counter += length  
                
                ref_seq_with_deletions_insertions.extend(reference_sequence[ref_counter:ref_counter + length]) 
                ref_counter += length  

            elif op == "I":  
                sequence.extend(entry['sequence'][counter:counter + length])  
                signal.append(signals[counter])
                counter += length  
                
                ref_seq_with_deletions_insertions.extend([''] * length)  

            elif op == "D":  
                sequence.extend([''] * length)  
                signal.append([None] * length * entry['stride'])  # it means there is no signal due to deletion
                
                ref_seq_with_deletions_insertions.extend(reference_sequence[ref_counter:ref_counter + length])  
                ref_counter += length  
                            
            else:  
                warnings.warn(f"Invalid CIGAR operator: '{op}'", UserWarning)  
                break  

        result.append({
            'sequence': np.array(sequence),
            'ref_seq': np.array(ref_seq_with_deletions_insertions),
            'signal': np.array(signal, dtype='object')
        })

    return result

def test_basecall_accuracy(result_dict):
    pct=[]
    for i in range(len(result_dict)):
        matching, not_matching = 0,0
        for s, r in zip(result_dict[i]['sequence'], result_dict[i]['ref_seq']):
            if not s=='' and not r=='' :
                if s==r:
                    matching +=1 
                else: 
                    not_matching+=1
        
        pct.append(matching/ (matching+not_matching))
    print(f'Min accuracy:  {100*min(pct):.3f}%')
    print(f'Max accuracy:  {100*max(pct):.3f}%')
    print(f'Mean accuracy: {100*np.mean(pct):.3f}%')
    print(f'Std accuracy:  {100*np.std(pct):.3f}%')
    return None"""

---
# Reference file

In [3]:
with open("/v/volumes/nanopore/ref/p300_ID3.fasta", 'r') as file:
    p300_ID3_ref = file.read().split('\n')[1]
p300_ID3_ref = np.array(list(p300_ID3_ref))
print(f'Length of ref sequence: {len(p300_ID3_ref)}')
p300_ID3_ref

Length of ref sequence: 1164


array(['A', 'A', 'C', ..., 'C', 'A', 'G'], dtype='<U1')

In [4]:
def find_the_greatest_neighbors(dst):
    'Squared geometric mean'
    geom_mean_sq = []
    for i in range(len(dst)-1):
        geom_mean_sq.append(dst[i]*dst[i+1] )
    return np.argmax(geom_mean_sq)+1

T_positions = np.where(p300_ID3_ref == 'T')[0]
T_distances = np.diff(T_positions)
T_max_dist_pos = find_the_greatest_neighbors(T_distances)
T_max_pos = T_positions[T_max_dist_pos]
print(f"The position: {T_max_pos}")
print(f"Distances: {T_distances[T_max_dist_pos-2: T_max_dist_pos+2]}")
p300_ID3_ref[T_max_pos-15:T_max_pos+21]

The position: 692
Distances: [ 6 15 20  1]


array(['T', 'C', 'C', 'A', 'A', 'G', 'G', 'C', 'A', 'G', 'A', 'C', 'A',
       'C', 'C', 'T', 'A', 'C', 'A', 'C', 'C', 'A', 'C', 'C', 'A', 'A',
       'C', 'A', 'A', 'C', 'A', 'C', 'A', 'A', 'C', 'T'], dtype='<U1')

---
# Thymine dataset

In [5]:
T_sam_data = read_sam(sorted(glob("/v/projects/nanopore/balazs/data/thymine_0421/*0.sam")),
                      min_length=len(p300_ID3_ref)*0.8,
                      max_length=len(p300_ID3_ref)*1.2, 
                      min_MAPQ=50,
                      max_insertion=7, 
                      max_deletion=7,
                      max_indel_frequency=0.4,
                      #verbose=True # Number of reads stored: 17420
                     )

NameError: name 'read_sam' is not defined

In [6]:
T_forward_strands = search_in_data(T_sam_data, "sam_flag",  0, only_first_match=False)
#T_reverse_strands = search_in_data(T_sam_data, "sam_flag", 16, only_first_match=False)

NameError: name 'search_in_data' is not defined

In [ ]:
path_to_Thymine_fast5 = '/v/projects/nanopore/agnes/signal_visualization/data/timin0421/workspace/'

thymine_data2 = get_raw_signal_with_bases(T_forward_strands[:100], path_to_Thymine_fast5, reference_sequence=p300_ID3_ref, verbose=0)

In [ ]:
---

In [ ]:
thymine_data

In [ ]:
test_basecall_accuracy(thymine_data)

In [ ]:
test_basecall_accuracy(thymine_data2)

---
# Uracil dataset

In [ ]:
U_sam_data = read_sam(sorted(glob("/v/projects/nanopore/balazs/data/uracil_0504/*0.sam")[:7]),
                      min_length=len(p300_ID3_ref)*0.8,
                      max_length=len(p300_ID3_ref)*1.2, 
                      min_MAPQ=40,
                      max_insertion=7, 
                      max_deletion=7,
                      max_indel_frequency=0.4,
                      #verbose=True # Number of reads stored: 17420
                     )

In [ ]:
U_forward_strands = search_in_data(U_sam_data, "sam_flag",  0, only_first_match=False)
#U_reverse_strands = search_in_data(U_sam_data, "sam_flag", 16, only_first_match=False)

In [ ]:
path_to_Uracil_fast5 = '/v/projects/nanopore/agnes/signal_visualization/uracil0504/guppy/workspace/'

uracil_data = get_raw_signal_with_bases(U_forward_strands[:100], path_to_Uracil_fast5, reference_sequence=p300_ID3_ref, verbose=0)

In [ ]:
test_basecall_accuracy(uracil_data)

---
---

```python
## Violin plot

import seaborn as sns
import matplotlib.pyplot as plt

# Sample data
import numpy as np
import pandas as pd

np.random.seed(10)
df = pd.DataFrame({
    "Category": np.tile(["A", "B", "C"], 50),
    "Value": np.concatenate([np.random.normal(loc=mu, scale=1, size=50) for mu in [5, 6, 7]])
})

# Violin plot with reduced width
plt.figure(figsize=(6, 4))
sns.violinplot(x="Category", y="Value", data=df, width=1)  # Reduce width to make violins narrower
plt.show()
```